# Growth requirements

This notebook is focused on carbon sources, minimal media, uptakes etc. of the models present in this repository.

*Author: Famke Baeuerle*

In [3]:
import refinegems as rg
from cobra.medium import minimal_medium
import pandas as pd
import numpy as np

CARBON = ["EX_gly_e", "EX_fum_e", "EX_male_e", "EX_pyr_e", "EX_succ_e", "EX_glc__D_e", "EX_urea_e", "EX_22bipy_e", "EX_boricacid_e", 
            "EX_adocbl_e", "EX_4abz_e", "EX_pnto__R_e", "EX_pydam_e", "EX_nac_e", 'EX_fru_e', 'EX_succ_e', 'EX_gam_e', 'EX_glyald_e']
NITROGEN = ["EX_ala__L_e", "EX_arg__L_e", "EX_cys__L_e", "EX_glu__L_e", "EX_his__L_e", "EX_leu__L_e", "EX_lys__L_e", "EX_orn_e", "EX_phe__L_e", 
            "EX_pro__L_e", "EX_ser__L_e", "EX_thr__L_e", "EX_trp__L_e", "EX_val__L_e", "EX_cit_e", "EX_btn_e", "EX_ribflv_e", "EX_thm_e"]

In [ ]:
def remove_common(a, b):
    a, b = [i for i in a if i not in b], [j for j in b if j not in a]
    print("list1 : ", a)
    print("list2 : ", b)

In [ ]:
common_TS=['EX_ca2_e',
 'EX_cl_e',
 'EX_cobalt2_e',
 'EX_cu2_e',
 'EX_k_e',
 'EX_mg2_e',
 'EX_mn2_e',
 'EX_o2_e',
 'EX_pi_e',
 'EX_zn2_e']

## Minimal media for all models

First we load all models with refineGEMs.

In [4]:
modelpaths = ['../../models/iCstr1054FB23.xml', '../../models/iCstr1197FB23.xml', '../../models/iCstr1115FB23.xml', '../../models/iCstr1116FB23.xml']
cmods = [rg.io.load_model_cobra(path) for path in modelpaths]

Then we apply the cobrapy minimal_medium function to all models wth 4 components.

In [5]:
for model in cmods:
    mini = minimal_medium(model, 0.1, minimize_components=4, open_exchanges=True)
    mini.to_csv('../../analysis/minmal_media/Cstr_' + model.id[4:] + '_minmed.csv')

### Growth behavior on minimal media

In [17]:
model = cmods[0]
col = '3' # decide for one of the columns (0 to 3)
min_medium = pd.read_csv('../../analysis/minmal_media/Cstr_TS_minmed.csv')
min_medium = min_medium[['Unnamed: 0', col]]
min_medium = min_medium[min_medium[col] != 0]
min_medium = min_medium.set_index('Unnamed: 0').to_dict()[col]
print(min_medium)

print('Investigate minimal medium of strain ' + model.id)
with model:
    model.medium = min_medium
    growth = model.slim_optimize()
    time = (np.log(2) / growth) * 60
    print('Doubling time with standard fluxes:')
    print(time)

for key, value in min_medium.items():
    min_medium[key] = 10.0

with model:
    model.medium = min_medium
    growth = model.optimize() 
    time = (np.log(2) / growth.objective_value) * 60
    print('Doubling time with 10.0 as flux:')
    print(time)

{'EX_2pglyc_e': 0.0986092999917973, 'EX_ca2_e': 0.0005205, 'EX_chols_e': 0.0250625999998419, 'EX_cl_e': 0.0005205, 'EX_cobalt2_e': 1e-05, 'EX_cu2_e': 7.09e-05, 'EX_glyglygln_e': 135.2566503614603, 'EX_k_e': 0.0195193, 'EX_mg2_e': 0.0008675, 'EX_mn2_e': 6.91e-05, 'EX_o2_e': 860.4240884351017, 'EX_salchs4fe_e': 0.2942146547067068, 'EX_zn2_e': 3.41e-05}
Investigate minimal medium of strain iCstr1054FB23
Doubling time with standard fluxes:
415.8883084145069
Doubling time with 10.0 as flux:
930.3585923974731


## Carbon sources

In [25]:
def simulate_carbon_sources(model, base_medium):
    carbon_sources = CARBON
    simulate = {}
    with model:
        simulate['default uptake'] = model.optimize().objective_value
    with model:
        model.medium = {i: 10.0 for i in base_medium}
        #print(model.medium)
        remove = []
        #model.reactions.get_by_id('EX_o2_e').lower_bound = 0.
        # does not grow without oxygen
        carbon_sources.extend(NITROGEN)
        for source in carbon_sources:
            try:
                model.reactions.get_by_id(source).lower_bound = 0.
            except (KeyError):
                print(source + ' not in model')
                remove.append(source)
        print('Minimal medium to simulate on: ')
        print(model.medium)
        carbon_sources = [x for x in carbon_sources if x not in remove]
        simulate['no carbon'] = model.optimize().objective_value
        for source in carbon_sources:
            model.reactions.get_by_id(source).lower_bound = -10.
            solution = model.optimize().objective_value
            simulate[source] = solution#(np.log(2) / solution) * 60
            model.reactions.get_by_id(source).lower_bound = 0.
            
    return simulate

We take the minimal uptake from the calculated minimal medium as base medium to simulate the carbon sources on.

In [71]:
model = cmods[3] 
col = '0' # decide for one of the columns (0 to 3)
minmed = pd.read_csv('../../analysis/minmal_media/Cstr_1116_minmed.csv')
minmed = minmed[['Unnamed: 0', col]]
minmed = minmed[minmed[col] != 0]
minmed = minmed.set_index('Unnamed: 0').to_dict()[col]
minmed['EX_pnto__R_e'] = 10.0
minmed

{'EX_ca2_e': 0.0005205,
 'EX_cl_e': 0.0005205,
 'EX_cobalt2_e': 1e-05,
 'EX_cu_e': 7.09e-05,
 'EX_g3pe_e': 31.64972680157348,
 'EX_gthrd_e': 79.24542397060202,
 'EX_istfrnA_e': 0.0014523000000295,
 'EX_k_e': 0.0195193,
 'EX_mg2_e': 0.0008675,
 'EX_mn2_e': 6.91e-05,
 'EX_nmn_e': 0.0002278000001751,
 'EX_o2_e': 697.8102169681671,
 'EX_pnto__R_e': 10.0,
 'EX_so4_e': 0.0004338,
 'EX_zn2_e': 3.41e-05}

Test if nitrogen or carbon source is in the minmal medium composition.

In [68]:
n = {k: minmed[k] for k in minmed.keys() & set(NITROGEN)}
n

{}

In [72]:
print('Simulate carbon sources of strain ' + model.id)
carbon = simulate_carbon_sources(model, minmed.keys())

Simulate carbon sources of strain iCstr1116FB23
EX_male_e not in model
EX_22bipy_e not in model
EX_boricacid_e not in model
EX_adocbl_e not in model
EX_4abz_e not in model
EX_pydam_e not in model
EX_nac_e not in model
EX_orn_e not in model
EX_ribflv_e not in model
EX_orn_e not in model
EX_ribflv_e not in model
EX_orn_e not in model
EX_ribflv_e not in model
EX_orn_e not in model
EX_ribflv_e not in model
EX_orn_e not in model
EX_ribflv_e not in model
EX_orn_e not in model
EX_ribflv_e not in model
EX_orn_e not in model
EX_ribflv_e not in model
EX_orn_e not in model
EX_ribflv_e not in model
EX_orn_e not in model
EX_ribflv_e not in model
EX_orn_e not in model
EX_ribflv_e not in model
EX_orn_e not in model
EX_ribflv_e not in model
EX_orn_e not in model
EX_ribflv_e not in model
EX_orn_e not in model
EX_ribflv_e not in model
Minimal medium to simulate on: 
{'EX_ca2_e': 10.0, 'EX_cl_e': 10.0, 'EX_cobalt2_e': 10.0, 'EX_cu_e': 10.0, 'EX_g3pe_e': 10.0, 'EX_gthrd_e': 10.0, 'EX_istfrnA_e': 10.0, 'EX

Now we create a list of carbon sources that lead to the highest growth rates for each of the strains.

In [73]:
for source, value in carbon.items():
    if value > 0.0009:
        print(source, round(value, 4))

default uptake 34.5655
no carbon 0.0
EX_gly_e 0.0
EX_fum_e 0.0
EX_pyr_e 0.0
EX_succ_e 0.0
EX_glc__D_e 0.0
EX_urea_e 0.0
EX_pnto__R_e 0.0876
EX_fru_e 0.0
EX_gam_e 0.0
EX_glyald_e 0.0
EX_ala__L_e 0.0
EX_arg__L_e 0.0
EX_cys__L_e 0.0
EX_glu__L_e 0.0
EX_his__L_e 0.0
EX_leu__L_e 0.0
EX_lys__L_e 0.0
EX_phe__L_e 0.0
EX_pro__L_e 0.0
EX_ser__L_e 0.0
EX_thr__L_e 0.0
EX_trp__L_e 0.0
EX_val__L_e 0.0
EX_cit_e 0.0
EX_btn_e 0.0
EX_thm_e 0.0


Now we extract minimal compositions for anaerobic growth.

In [114]:
minmed = pd.read_csv('../../analysis/minmal_media/Cstr_1116_minmed.csv')
minmed = minmed[minmed['1'] != 0]
minmed = minmed.set_index('Unnamed: 0').to_dict()['1']
pd.Series([key[3:-2] for key in minmed.keys()]).to_clipboard()

**Result of the anaerobic minimal medium for each strain**

| TS      | 1197    | 1115    | 1116    |
|---------|---------|---------|---------|
| ca2     | ca2     | acglu   | ca2     |
| cgly    | cl      | ca2     | cl      |
| chols   | cobalt2 | cl      | cobalt2 |
| cl      | cu      | cobalt2 | cu      |
| cobalt2 | gthrd   | cu2     | fe2     |
| cu2     | h2o2    | glyc3p  | g3pe    |
| h2o2    | istfrnA | h2o2    | gthrd   |
| istfrnB | k       | istfrnA | h2o2    |
| k       | mg2     | k       | k       |
| mg2     | mn2     | mg2     | mg2     |
| mn2     | pi      | mn2     | mn2     |
| pi      | pnto__R | pnto__R | nmn     |
| zn2     | zn2     | s       | pnto__R |
|         |         | so4     | so4     |
|         |         | zn2     | zn2     |

## Essential reactions

In [116]:
model = cmods[0]
ess = []
for reaction in model.reactions:
    with model as model:
        reaction.knock_out()
        model.optimize()
        if model.objective.value <= 11:
            #print('%s blocked (bounds: %s), new growth rate %f $' % (reaction.id, str(reaction.bounds), model.objective.value))
            ess.append(reaction.id)
ess

['ADPT',
 'ALAALAr',
 'ASAD',
 'ASPK',
 'CA2abcpp',
 'CA2tex',
 'CDPMEK',
 'CHORS',
 'CLtex',
 'CLtipp',
 'Cuabc',
 'DAPE',
 'DHDPRy',
 'DHDPS',
 'DHNAOT',
 'DMATT',
 'DPCOAK',
 'DTMPK',
 'DXPRIi',
 'DXPS',
 'FMNAT',
 'GK1',
 'GRTT',
 'ICHORS',
 'Kt2r',
 'MECDPS',
 'MEPCT',
 'METAT',
 'MGt5',
 'MNabc',
 'NADK',
 'NADS1',
 'NDPK1',
 'NDPK2',
 'NDPK4',
 'NNATr',
 'NNDPR',
 'NPHS',
 'OCTDPS',
 'PAPPT3',
 'PNTK',
 'PPA',
 'PPNCL3',
 'PRPPS',
 'PSCVT',
 'PTPATi',
 'QULNS',
 'RBFK',
 'SDPDS',
 'SDPTA',
 'SEPHCHCS',
 'SHCHCS3',
 'SHKK',
 'SUCBZL',
 'SUCBZS',
 'THDPS',
 'UAAGDS',
 'UAGCVT',
 'UAGDP',
 'UAGPT3',
 'UAMAGS',
 'UAMAS',
 'UAPGR',
 'UGMDDS',
 'UMPK',
 'EX_ca2_e',
 'EX_cl_e',
 'EX_cobalt2_e',
 'EX_cu2_e',
 'EX_k_e',
 'EX_mg2_e',
 'EX_mn2_e',
 'EX_zn2_e',
 'Growth']

## Fluxes on different growth media

In [169]:
# we have to look systematically at the flux distribution
m9 = pd.read_csv('../../data/media_db.csv')[pd.read_csv('../../data/media_db.csv')['medium']=='M9']['BiGG'].to_list()
cgx = pd.read_csv('../../data/media_db.csv')[pd.read_csv('../../data/media_db.csv')['medium']=='CGXII']['BiGG'].to_list()
lb = pd.read_csv('../../data/media_db.csv')[pd.read_csv('../../data/media_db.csv')['medium']=='LB']['BiGG'].to_list()
rpmi = pd.read_csv('../../data/media_db.csv')[pd.read_csv('../../data/media_db.csv')['medium']=='RPMI']['BiGG'].to_list()[:-1]
rpmi.remove('ncam')
rpmi.remove('pydxn')
rpmi.remove('cbl1')
rpmi.remove('4hpro_LT')
rpmi.remove('b12')
lb.remove(np.nan)
lb.remove(np.nan)
lb.remove('pydx')
lb.remove('cbl1')
lb.remove('cbl2')
lb.remove('b12')
lb.remove('cd2')
lb.remove('hg2')
lb.remove('ins')
lb.remove('thymd')
lb.remove('uri')
lb.remove('dad_2')
lb.remove('adn')
lb.remove('chor')
lb.remove('cmp')
lb.remove('cro4')
lb.remove('gmp')
lb.remove('gsn')
lb.remove('nac')
lb.remove('ump')

with model:
    model.medium = {'EX_' + i + '_e':10 for i in m9}
    flux_m9 = pd.DataFrame(model.optimize().fluxes).rename({'fluxes':'fluxes_m9'}, axis=1)

with model:
    model.medium = {'EX_' + i + '_e':10 for i in lb}
    flux_lb = pd.DataFrame(model.optimize().fluxes).rename({'fluxes':'fluxes_lb'}, axis=1)
    
with model:
    model.medium = {'EX_' + i + '_e':10 for i in rpmi}
    flux_rpmi = pd.DataFrame(model.optimize().fluxes).rename({'fluxes':'fluxes_rpmi'}, axis=1)
    
with model:
    model.medium = {'EX_' + i + '_e':10 for i in cgx}
    flux_cgx = pd.DataFrame(model.optimize().fluxes).rename({'fluxes':'fluxes_cgx'}, axis=1)

all_flux = flux_rpmi.join(flux_m9).join(flux_lb).join(flux_cgx)
flux_rpmi[flux_rpmi['fluxes_rpmi'] != 0]

['glc__D', 'ala__L', 'asp__L', 'glu__L', 'his__L', 'leu__L', 'met__L', 'pro__L', 'thr__L', 'tyr__L', 'arg__L', 'cys__L', 'gly', 'ile__L', 'lys__L', 'phe__L', 'ser__L', 'trp__L', 'val__L', 'thm', 'ribflv', 'pnto__R', 'btn', 'fol', 'na1', 'cl', 'so4', 'k', 'pi', 'ca2', 'mg2', 'zn2', 'aso3', 'hxan', 'dcyt', 'ura', 'o2', 'h', 'h2o', 'amp', 'cobalt2', 'cu2', 'fe2', 'fe3', 'h2s', 'lipoate', 'mn2', 'mobd', 'nh4', 'ni2', 'pheme']


,fluxes_rpmi
ACLDC,1.280775e-12
ACLS_d,-2.255211e-14
ACLSb,1.347431e-12
ACONT,-1.437305e-13
ACTNabc1,1.280775e-12
...,...
EX_trp__L_e,-4.412399e-15
EX_tyr__L_e,-1.070408e-14
EX_zn2_e,-2.646989e-17
Growth,7.762431e-14


Looking at the flux distribution.

In [176]:
all_flux.loc[~(all_flux != 0).any(axis=1)] # 1578 reactions have no flux
all_flux.loc[(all_flux != 0).any(axis=1)] # only 428 have flux
print(all_flux.loc[(all_flux > 0.005).all(axis=1)].round(2).to_latex())#.index 

\begin{tabular}{lrrrr}
\toprule
{} &  fluxes\_rpmi &  fluxes\_m9 &  fluxes\_lb &  fluxes\_cgx \\
\midrule
ALCD19     &       1000.0 &     1000.0 &    1000.00 &     1000.00 \\
ASPT       &        990.0 &     1000.0 &     990.04 &      996.83 \\
CMCBTFL    &       1000.0 &     1000.0 &    1000.00 &     1000.00 \\
CMCBTFU    &       1000.0 &     1000.0 &    1000.00 &     1000.00 \\
CO2t       &        990.0 &     1000.0 &     988.09 &      988.78 \\
FUM        &       1000.0 &     1000.0 &    1000.00 &     1000.00 \\
Kt2r       &        485.0 &      500.0 &     485.54 &      492.32 \\
Kt3r       &        485.0 &      500.0 &     485.44 &      492.32 \\
MDH        &       1000.0 &     1000.0 &    1000.00 &     1000.00 \\
NAt3\_1     &        990.0 &     1000.0 &     990.63 &     1000.00 \\
SUCTARTtpp &       1000.0 &     1000.0 &    1000.00 &     1000.00 \\
r1143      &        990.0 &     1000.0 &     990.63 &     1000.00 \\
\bottomrule
\end{tabular}



## Flux variability analysis

In [ ]:
model.optimize()
model.summary(fva=0.9)